<center>
<img src="http://corpuslg.org/lael_english/wp-content/uploads/2020/04/lael_50_years_narrow_white.png.400px_300dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Images function set

## Importing the required libraries

In [1]:
import subprocess
from IPython.display import clear_output
import os
from pathlib import Path
from google.cloud import storage
from google.cloud import vision

## Preamble

- Group number definition
- Selects the syntax of the the commands 'shuf' and 'sed' for Mac or Linux platforms
- The variables 'myshuf' and 'mysed' are not used in the programme

In [ ]:
#enter your group number
group=4

#enter the label for your system, mac or linux
mysystem=mac
#mysystem=linux

  if [ "$mysystem" == mac ]
    then
    myshuf=gshuf
    mygsed=gsed
  else
    myshuf=shuf
    mygsed=sed
  fi 


## presample function

- Processes 'tweets/scraped.txt' into 'images/presample.txt'
- Correction to create the folder 'images' for the first tine

### Original function

In [ ]:
presample () {

mkdir -p images

grep fullUrl tweets/scraped.txt | cut -f2- | nl | sed 's/^[ ]*//' > images/presample.txt

}


### Executable from this notebook

In [13]:
subprocess.run(['wsl', './presample.sh'], capture_output = True)

CompletedProcess(args=['wsl', './presample.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J', stderr=b'')

## collecturls function

- Processes 'images/presample.txt' into 'images/images_index.txt'
- Defines 13 image batches with 1000 images in each batch
- Creates one folder for each batch in 'images/images'
- The commands 'rg' and 'jq' are required
- Correction to resolve folder naming conflict with 'grabimages' function
    - from: 'sort z | uniq > folders'
    - to: 'sort z | uniq | sed 's/fl://' > folders'

### Original function

In [ ]:
getimagesurls () {

last=$( cat images/presample.txt | wc -l | tr -dc '[0-9]' )

#mkdir -p images

rm -f images/urls.txt  ### WATCH THIS!

for i in $(eval echo {1..$last});
do
        rg -m1 "^"$i"	" images/presample.txt | jq '.' > z
        file=$( grep -m1 'fullUrl' z | cut -d'"' -f4  )
        format=$( echo $file | tr '?&=' '|' | cut -d'|' -f3 )
        id=$( grep -m1 'id"' z | tr '~' ' ' | sed -e 's/^[ ]*//' -e 's/:/~/' | cut -d'~' -f2 | tr -dc '[0-9]' )
        username=$( grep -m1 'username"' z | tr '~' ' ' | sed -e 's/^[ ]*//' -e 's/:/~/' | cut -d'~' -f2 | sed -e 's/"//' -e 's/^[ ]*//' -e 's/",$//' | tr '[:upper:]' '[:lower:]' )
        date=$( grep -m1 'date"' z | tr '~' ' ' | sed -e 's/^[ ]*//' | cut -d'~' -f2 | cut -d'T' -f1 | tr -dc '[0-9-]' )

        echo "---- collecturls $i / $last ----"

        echo "id:"$id"|d:$date|u:"$username"|i:$file|f:$format" >> images/urls.txt
done 

grep 'id:...................|' images/urls.txt | nl -nrz | sed 's/^/t:/' | tr '\t' '|' > w

for n in `seq -w 1 13`  # how many batches
do
  printf "$n\n%.0s" {1..1000}  # how many lines in each batch
done | sed 's/^/fl:/' > z

paste z w | tr '\t' '|' | grep 'id:' > images/images_index.txt

#sort z | uniq > folders
sort z | uniq | sed 's/fl://' > folders
while read folder
do
    mkdir -p images/images/$folder
done < folders

}


### Executable from this notebook

In [14]:
subprocess.run(['wsl', './getimagesurls.sh'], capture_output = True)

CompletedProcess(args=['wsl', './getimagesurls.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J---- collecturls 1 / 2 ----\n---- collecturls 2 / 2 ----\n', stderr=b'')

## grabimages function

- Processes 'images/images_index.txt' into ''
- The command 'curl' is required

### Original function

In [ ]:
grabimages () {

last=$( cat images/images_index.txt | wc -l | tr -cd '[0-9]' )

for i in $(eval echo {1..$last});
do
    sed -n "$i"p images/images_index.txt > z
    folder=$( cut -d'|' -f1 z | sed 's/fl://' )
    n=$( cut -d'|' -f2 z | sed 's/t://' )
    id=$( cut -d'|' -f3 z | sed 's/id://' )
    file=$( cut -d'|' -f6 z | sed 's/i://' )
    ext=$( cut -d'|' -f7 z | sed 's/f://' )
    
    echo "---- fetching image $n / $last ----"
    
    curl -k "$file" > images/images/"$folder"/"$n"."$ext"
done 

}


### Executable from this notebook

In [15]:
subprocess.run(['wsl', './grabimages.sh'], capture_output = True)

CompletedProcess(args=['wsl', './grabimages.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J---- fetching image 000001 / 2 ----\n---- fetching image 000002 / 2 ----\n', stderr=b'  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current\n                                 Dload  Upload   Total   Spent    Left  Speed\n\r  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\r100 80693  100 80693    0     0   496k      0 --:--:-- --:--:-- --:--:--  498k\n  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current\n                                 Dload  Upload   Total   Spent    Left  Speed\n\r  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\r  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0\r100 82431  100 82431    0     0   653k      0 --:--:-- --:--:-- --:--:--  649k\n')

## removedupes function

- Removes duplicate images posted in the same message or in repeated messages by the same user (same message id)
- The function was empirically tested by manually introducing a duplicate image-containing message in 'tweets/scraped.txt' and it works as expected. It returns the following error message that apparently does not harm its functionality
    - './removedupes.sh: line 27: remove: No such file or directory\ngrep: remove: No such file or directory\nfind: paths must precede expression: `empty'\n'
- When there are no duplicates, the function ends with the following error message. This result is not harmful because it proves that there are no duplicates
    - './removedupes.sh: line 27: remove: No such file or directory\ngrep: remove: No such file or directory\nfind: paths must precede expression: `empty'\n'
- Correction to avoid 'images/images_index.txt' to have all lines excluded in case thare are no duplicates and, thus, the 'remove' file is empty

### Original function

In [ ]:
removedupes () {

# remove duplicate images posted in the same message or in repeated messages by the same user (same message id)
    
cut -d'|' -f3 images/images_index.txt | sed 's/id://' | sort | uniq -c | grep -v ' 1 ' | nl | sed 's/^[ ]*//' | tr '\t' ' ' | tr -s ' ' > d

rm -f remove
while read n maxhits id
do
    echo "--- listing $n ---"
    grep -m$maxhits $id images/images_index.txt | cut -d'|' -f6 | sort | uniq -d | sed 's/i://' > dupes
    
    while read dupe
    do
        grep $dupe images/images_index.txt | tail +2 | cut -d'|' -f2
    done < dupes >> remove
done < d 

# remove duplicate image files
while read dupe
do
    pretty=$( echo $dupe | sed 's/t://' )
    folder=$( grep $dupe images/images_index.txt | cut -d'|' -f1 | sed 's/fl://' )
    ext=$( grep $dupe images/images_index.txt | cut -d'|' -f7 | sed 's/f://' )
    rm -f images/images/"$folder"/"$pretty"."$ext"
    echo "--- removing images/images/"$folder"/"$pretty"."$ext" ---"
done < remove 

# remove dupes from index
#grep -vf remove images/images_index.txt > z ; mv z images/images_index.txt
if [[ -s remove ]]; then
    grep -vf remove images/images_index.txt > z ; mv z images/images_index.txt
fi

# remove dupes from index again, for some reason some still remain
cut -d'|' -f2  images/images_index.txt | cut -d':' -f2 > i
find images/images -type f | cut -d'/' -f4 | cut -d'.' -f1 > f
cat i i f | sort | uniq -c | grep ' 2 ' | cut -c6- | grep -v '^$' | sed 's/^/t:/' > ionly
grep -vf ionly images/images_index.txt > b ; mv b images/images_index.txt

# remove empty image files
find images/images -type f empty -exec rm {} +

# remove empty image files from index
cut -d'|' -f2  images/images_index.txt | cut -d':' -f2 > i
find images/images -type f | cut -d'/' -f4 | cut -d'.' -f1 > f
cat i i f | sort | uniq -c | grep ' 2 ' | cut -c6- | grep -v '^$' | sed 's/^/t:/' > ionly
grep -vf ionly images/images_index.txt > b ; mv b images/images_index.txt

}


### Executable from this notebook

In [16]:
subprocess.run(['wsl', './removedupes.sh'], capture_output = True)

CompletedProcess(args=['wsl', './removedupes.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J', stderr=b"./removedupes.sh: line 27: remove: No such file or directory\nfind: paths must precede expression: `empty'\n")

## uploadtobucket function

- Copies the 'images/images' folder to the user's Google Cloud Storage bucket

### Original function

In [ ]:
uploadtobucket () {

# google cloud gsutil account = tonyberber@gmail.com

gcloud alpha storage cp -R images/images gs://socialmediaclassimages/group

}


### Executable from this notebook

- On your Desktop, click on 'Google Cloud SDK Shell'
- On the terminal window execute the commands as follows:

#### Using Google Cloud SDK Shell

In [ ]:
# Go to the folder where the 'images' folder is located
C:\Users\eyamr\AppData\Local\Google\Cloud SDK>cd C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground
C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground>dir
 Volume in drive C is OS
 Volume Serial Number is FA6F-4E60

 Directory of C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground

20/09/2023  17:43    <DIR>          .
19/09/2023  15:29    <DIR>          ..
<omitted>
20/09/2023  17:37    <DIR>          images
<omitted>
              24 File(s)         51.289 bytes
               5 Dir(s)  134.018.973.696 bytes free

# List the buckets
C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground>gcloud storage ls
gs://laelimages/

# Run the following commad
C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground>gcloud alpha storage cp -R images/images gs://laelimages
Copying file://images\images\01\000001.jpg to gs://laelimages/images/01/000001.jpg
Copying file://images\images\01\000002.jpg to gs://laelimages/images/01/000002.jpg
  Completed files 2/2 | 159.3kiB/159.3kiB

Average throughput: 6.0MiB/s

C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground>

#### Using the 'google-cloud-storage' Python library
Alternative 1: Completely equivalent to the Google Cloud SDK approach

In [6]:
def upload_directory_to_bucket(bucket_name, source_directory, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    for root, dirs, files in os.walk(source_directory):
        for file in files:
            local_path = os.path.join(root, file)
            blob_path = os.path.join(destination_blob_name, os.path.relpath(local_path, source_directory)).replace('\\', '/')
            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_path)

    print(f'Directory {source_directory} uploaded to {bucket_name}/{destination_blob_name} successfully!')

end = False
while end == False:
    my_bucket = str(input('Enter your bucket name: '))
    if my_bucket != '':
        bucket_name = my_bucket
        end = True
        clear_output()

source_directory = r'.\images\images'
destination_blob_name = 'images'
upload_directory_to_bucket(bucket_name, source_directory, destination_blob_name)

Directory .\images\images uploaded to laelimages/images successfully!


#### Using the 'google-cloud-storage' Python library
Alternative 2: Copying even empty subdirectories

In [ ]:
def upload_directory_to_bucket(bucket_name, source_directory, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    for root, dirs, files in os.walk(source_directory):
        for file in files:
            local_path = os.path.join(root, file)
            blob_path = os.path.join(destination_blob_name, os.path.relpath(local_path, source_directory)).replace('\\', '/')
            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_path)

        for dir in dirs:
            dir_path = os.path.join(destination_blob_name, os.path.relpath(os.path.join(root, dir), source_directory)).replace('\\', '/')
            blob = bucket.blob(dir_path + '/')
            blob.upload_from_string('')

    print(f'Directory {source_directory} uploaded to {bucket_name}/{destination_blob_name} successfully!')

end = False
while end == False:
    my_bucket = str(input('Enter your bucket name: '))
    if my_bucket != '':
        bucket_name = my_bucket
        end = True
        clear_output()

source_directory = r'.\images\images'
destination_blob_name = 'images'
upload_directory_to_bucket(bucket_name, source_directory, destination_blob_name)

## googlelabels function

- description

### Original function

In [ ]:
googlelabels () {

# speed: 1 file per second

while read folder
do
  mkdir -p images/google_cloud/labels/$folder
  ### rm -f images/google_cloud/labels/$folder/*
done < folders

last=$( tail -1 images/images_index.txt | cut -d'|' -f2 | sed 's/t://'  )

for i in $(eval echo {1..$last});
do
    sed -n "$i"p images/images_index.txt > z
    folder=$( cut -d'|' -f1 z | sed 's/fl://' )
    n=$( cut -d'|' -f2 z | sed 's/t://' )
    id=$( cut -d'|' -f3 z | sed 's/id://' )
    file=$( cut -d'|' -f6 z | sed 's/i://' )
    ext=$( cut -d'|' -f7 z | sed 's/f://' )
    
    echo "---- detect-labels $i / $last ----"
    
    gcloud ml vision detect-labels --max-results=150  gs://socialmediaclassimages/group/"$folder"/"$n"."$ext" > images/google_cloud/labels/"$folder"/"$n".txt
    
done 

}


### Executable from this notebook

In [25]:
def detect_labels(image_url):
    """Detects labels in the image URL using the Google Cloud Vision API."""
    client = vision.ImageAnnotatorClient()
    image = vision.Image(source=vision.ImageSource(image_uri=image_url))
    response = client.label_detection(image=image)
    if response.error.message:
        raise Exception(f'Error: {response.error.message}')
    return response.label_annotations

with open('folders', 'r') as f:
    for folder in f:
        folder = folder.strip()
        os.makedirs(f'images/google_cloud/labels/{folder}', exist_ok=True)
        # os.system(f'rm -f images/google_cloud/labels/{folder}/*')

last = subprocess.run(['wsl', 'tail', '-1', 'images/images_index.txt'], capture_output=True, text=True).stdout.strip().split('|')[1][2:]
last = int(last)

for i in range(1, last + 1):
    with open('images/images_index.txt', 'r') as f:
        line = next(line for j, line in enumerate(f, start=1) if j == i)
        folder = line.split('|')[0][3:]
        n = line.split('|')[1][2:]
        id = line.split('|')[2][3:]
        file = line.split('|')[5][2:]
        ext = line.split('|')[6][2:]

    print(f"---- detect-labels {i} / {last} ----")

    image_url = f'https://storage.cloud.google.com/{bucket_name}/images/{folder}/{n}.{ext}'
    labels = detect_labels(image_url)
    with open(f'images/google_cloud/labels/{folder}/{n}.txt', 'w') as f:
        for label in labels:
            f.write(f'{label.description}\n')

---- detect-labels 1 / 2 ----


Exception: Error: Bad image data.

## labeltypes function

- description

### Original function

In [ ]:
labeltypes () {

rm -f images/labels.txt

last=$( cat images/images_index.txt | wc -l | tr -cd '[0-9]' )

for i in $(eval echo {1..$last});
do    
    sed -n "$i"p images/images_index.txt > z
    folder=$( cut -d'|' -f1 z | sed 's/fl://' )
    n=$( cut -d'|' -f2 z | sed 's/t://' )
    id=$( cut -d'|' -f3 z | sed 's/id://' )
    date=$( cut -d'|' -f4 z | sed 's/d://' )
    username=$( cut -d'|' -f5 z | sed 's/u://' )
    
    echo "---- labeltypes $i / $last ---"
    
    grep  '"description":' images/google_cloud/labels/"$folder"/"$n".txt | cut -d':' -f2 | tr -d '"' | sed -e 's/^[ ]*//' | tr '\n' ' ' | sed 's/, $//' | tr ' ' '_' | sed 's/,_/,/g' | tr -d "'" | tr '[A-Z]' '[a-z]' | sed "s/^/fl:$folder|t:$n|id:$id|d:$date|u:$username|l:/" >> images/labels.txt
    echo >> images/labels.txt
done 

}


### Executable from this notebook

## toplabels function

- description

### Original function

In [ ]:
toplabels () {

cut -d'|' -f6 images/labels.txt | sed 's/l://' | tr ',' '\n' | sort | uniq -c | sort -nr | sed 's/^[ ]*//' | grep '[a-z]' | head -1000 | cut -d' ' -f2- | nl -nrz | sed 's/^/v/' | tr '\t' ' ' > images/selectedwords

cp images/selectedwords images/var_index.txt

}


### Executable from this notebook

## sas function

- description

### Original function

In [ ]:
sas () {

mkdir -p images/sas

rm -f images/columns

cut -d'|' -f3,6 images/labels.txt > a

while read n word 
do
  echo "--- $n ---"
  rg -w $word a | cut -d'|' -f1 | sed -e 's/id://' -e "s/$/ "$n" 1/" >> images/columns 
done < images/selectedwords

sort images/columns | uniq > a ; mv a images/columns  # to avoid words whose accents were stripped to be duplicated in the same text ; SAS can't handle that

#cut -d' ' -f2 tweets/selectedwords | gwc -L 
#head -1 columns | cut -d' ' -f1 | gwc -L

cp images/columns images/sas/data.txt

}


### Executable from this notebook

## datamatrix function

- description

### Original function

In [ ]:
datamatrix () {

mkdir -p images/temp

rm -f images/temp/*

cut -d' ' -f1 images/columns | uniq | sort > files

while read n word 
do
  echo "--- $n ---"
  rg -w $n images/columns | sort -t' ' -k1,1 > a
  echo "$n" > images/temp/$n
  join -a 1 -1 1 -2 1 -e 0 files a | sed "s/$/ $n 0/" | cut -d' ' -f3 >> images/temp/$n
done < images/selectedwords

echo "--- images/data.csv ...---"

awk '
        FNR==1 { col++ }
        FNR>max { max=FNR }
        { l[FNR,col]=$0 }
        END {
                for (i=1;i<=max;i++) {
                        for (j=1;j<=col;j++) {
                                printf "%-50s",l[i,j]
                        }
                        print ""
                }
        }
' images/temp/* > u
tr -s ' ' < u | tr ' ' ',' | sed 's/,$//' > images/data.csv

}


### Executable from this notebook

## correlationmatrix function

- description

### Original function

In [ ]:
correlationmatrix () {

echo "--- python correlation ... ---"

sed 's;data.csv;images/data.csv;' corr.py > p
python3 p > images/correlation

}


### Executable from this notebook

## formats function

- description

### Original function

In [ ]:
formats () {

nlines=$( cat tweets/emoji.txt | wc -l | tr -dc '[0-9]' )

tail +2 images/correlation | tr -s ' ' | sed 's/^/CORR /' > bottom
head -1 images/correlation | tr -s ' ' | sed 's/^[ ]*//' | sed "s/\(v......\)/$nlines/g" | sed 's/^/N . /' > n

sed 's;data.csv;images/data.csv;' std.py > p
python3 p > s 
tr -s ' ' < s | cut -d' ' -f2 | grep -v 'float' | tr '\n' ' ' | sed 's/^/STD	 . /' > std 
echo >> std

sed 's;data.csv;images/data.csv;' mean.py > p
python3 p > m 
tr -s ' ' < m | cut -d' ' -f2 | grep -v 'float' | tr '\n' ' ' | sed 's/^/MEAN . /' > mean
echo >> mean

cat mean std n bottom > images/sas/corr.txt

echo "PROC FORMAT library=work ;
  VALUE  \$lexlabels" > images/sas/word_labels_format.sas
tr '\t' ' ' < images/selectedwords | sed 's/\(.*\) \(.*\)/"\1" = "\2"/' | sed -e 's/&/and/g' -e 's/%/pc/g' >> images/sas/word_labels_format.sas
echo ";
run;
quit;" >> images/sas/word_labels_format.sas

}


### Executable from this notebook

## examples function

- description

### Original function

In [ ]:
examples () {

mkdir -p images/examples
rm -f images/examples/*

html2text -nobs images/sas/output_group"$group"_images/loadtable.html > a

rm -f x??
split -p'=====' a
ls x?? > files

while read file
do
      pole=$( grep '^Factor ' $file | cut -d' ' -f2,3 | sed -e 's/^/f/' -e 's/ //g' )
      sed 's/^[ ]*//' $file | grep '^[0-9]' | tr -dc '[:alpha:][:punct:][0-9]\n ' | sed 's/^/~/' | tr  '[:space:]()' ' ' | tr -s ' ' |  tr '~' '\n' | cut -d' ' -f2 | grep -v '^$' | sed "s/^/$pole /" 
done < files > images/examples/factors


rm -f x??

head -1 images/sas/output_group"$group"_images/group4_images_scores.tsv | tr -d '\r' | tr '\t' '\n' > vars
    
last=$( cut -d' ' -f1 images/examples/factors | tr -dc '[0-9\n]' | sort -nr | head -1 )
    
for i in $(eval echo {1..$last});
do
      column=$( echo " $i + 1 " | bc ) 
      cut -f1,"$column" images/sas/output_group4_images/group4_images_scores_only.tsv  | tail +2 > a

      for pole in pos neg
      do
        echo "--- "f"$i""$pole"" ---" 

        if [ "$pole" == pos ] ; then
           sort -nr -k2,2 a | grep -v '\-' | tr '\t' ' ' | grep -v ' 0' | head -20 | nl -nrz > files
        else
           sort -n -k2,2 a | grep '\-' | tr '\t' ' ' | grep -v ' 0' | grep -v '	0' | head -20 | nl -nrz > files
        fi

        grep f"$i""$pole" images/examples/factors | sort -t' ' -k2,2 | cut -d' ' -f2 | sort > factor_words
        
        while read n file score
        do

          grep -m1 $file images/sas/output_group4_images/group4_images_scores.tsv | tr -d '\r' | tr '\t' '\n' > scores
          paste vars scores | tr '\t' ' ' | grep '^v' | grep -v ' 0$' | cut -d' ' -f1 | sort > vars_text
          join vars_text images/selectedwords | cut -d' ' -f2 | sort > vars_text_codes
          username=$( grep -w $file user_index.txt | cut -d' ' -f2 )
          picture=$( grep -w $file images/images_index.txt | cut -d'|' -f2,7 | tr ':|' ' ' | cut -d' ' -f2,4 | sed 's;\(.*\) \(.*\);\1.\2;' )
          folder=$( grep -w $file images/images_index.txt | cut -d'|' -f1 | tr ':|' ' ' | cut -d' ' -f2  )
          url=$( grep -m1 -B5 $file tweets/jq.txt | grep '"url"'  | cut -d':' -f2- | tr -d '",' | sed 's/^[ ]*//' )
          extension=$( echo $picture | cut -d'.' -f2 )
          cp images/images/$folder/$picture images/examples/image_f"$i""$pole"_x`"$n"."$extension"

          echo "---------------" 

          echo "# $n" 
          echo "score = $score"  
          echo "url: $url"
          echo

          grep -w $file images/labels.txt | tr '|' '\n' | sed 's/l:/~/' | tr '~' '\n'    

          echo
          echo "Lemmas in this picture that loaded on the factor:"
          echo

          join vars_text_codes factor_words > ll
          tr '\n' ',' < ll | sed 's/,/, /g' | sed 's/, $//' > images/examples/lemmas_f"$i"_"$pole"_"$n".txt
          cat ll

          echo 

        done < files > images/examples/examples_f"$i"_"$pole".txt

      done

done

    #rm -f vars factor_words scores vars_text vars_text_codes

}


### Executable from this notebook